# Assignment 2

#### Student ID: *Double click here to fill the Student ID*

#### Name: *Double click here to fill the name*

## Q1: Data cleaning with the Melbourne Housing Dataset

In this question, you will practice data preparation skills that are often used in real-world projects.

The dataset is a snapshot of the [Melbourne Housing Market dataset](https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market) created by Tony Pino. It was scraped from publicly available results posted weekly on [Domain.com.au](https://www.domain.com.au/). The dataset includes the address, type of real estate, suburb, method of selling, number of rooms, price, real estate agent, date of sale, and distance from the central business district, among other variables.

Firstly, execute the following code snippet for data preparation (Remember to upload the dataset and import the dependent library first):

In [ ]:
# Function for comparing different approaches
def scoring_mape(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=2024)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_percentage_error(y_valid, preds)

# Prepare data
df = pd.read_csv('melb_data.csv')
y = df.Price

# To keep things simple, we'll split the columns into numerical can categorical features
data = df.drop(['Price', 'Date', 'Address'], axis=1)
num_col = data.select_dtypes(exclude=['object'])
cat_col = data.select_dtypes(exclude=['int64','float64'])

# Divide data into training and validation subsets, try to use the resulting datafram below in the following questions
X_train, X_valid, y_train, y_valid = train_test_split(data, y, train_size=0.8, test_size=0.2, random_state=2024)
# Numerical columns
X_train_num = X_train.select_dtypes(exclude=['object'])
X_valid_num = X_valid.select_dtypes(exclude=['object'])
# Categorical columns
X_train_cat = X_train.select_dtypes(exclude=['int64','float64'])
X_valid_cat = X_valid.select_dtypes(exclude=['int64','float64'])

To ensure reproducibility, please set all the random seeds to 2024:

### (a) Identify Columns with Missing Values and Calculate Their Missing Rates (5%)
Firstly, identify which columns contain missing values. Then, calculate the percentage of missing values in the dataset.

Hint: You should calculate the missing rates based on the original `data` matrix.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

### (b) Comparing Model Accuracy Using Three Data-Cleaning Approaches (10%)

To simplify the problem, we will consider only the numerical columns. Compare the model accuracy between the following three data-cleaning approaches:

1. **Removing all columns with missing values.**
2. **Replacing missing values with the median value of each column.**
3. **Using the iterative imputation method with a KNN regressor (15 neighbors) via [`IterativeImputer()`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html).**

Use the `scoring_mape()` function provided above to perform regression and calculate the Mean Absolute Percentage Error (MAPE) for each approach. Finally, provide comments on the results.

**Hint:** Since we are working with both training and validation sets, you should apply the same transformation when imputing missing values for both sets.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

### (c) Incorporating Categorical Variables (10%)

Now, we will consider categorical variables. Try to:

1. **Replace missing values with the most frequent value in each column for categorical variables.**
2. **Perform target encoding for the categorical variables.**
3. **Use the best approach you found in (b) to impute the numerical columns.**

Combine the above transformations, which include separate numerical and categorical pipelines, with the original training and validation data splits (`X_train` and `X_valid`). Then, use the `scoring_mape()` function to calculate the Mean Absolute Percentage Error (MAPE). Finally, provide comments on the results compared to those in part (b). (10%)

**Hint:** Since we are working with both training and validation sets, apply the same transformations when imputing missing data or encoding variables. You may find [ColumnTransformer](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html), [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), and be sure to enable [cross-fitting](https://scikit-learn.org/stable/modules/preprocessing.html#target-encoder) in target encoding.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

### (d) Identifying Label Issues with CleanLearning and Evaluating Model Accuracy (10%)

Use [`CleanLearning`](https://docs.cleanlab.ai/v2.5.0/cleanlab/regression/learn.html#cleanlab.regression.learn.CleanLearning) with a `RandomForestRegressor` to identify possible label issues in the training set. Then, fit `CleanLearning` on the preprocessed training dataset obtained in part (c) (i.e., after imputation and encoding). Finally, calculate the model's accuracy on the validation set. Provide comments on the results compared to those in part (c).

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

## Q2. Feature Engineering and Selection with the Ames Housing Dataset

In this question, we will examine several feature engineering and feature selection methods.

The dataset is a modified version of the Ames Housing Dataset. The original data was compiled by Dean De Cock for use in data science education and published in [De Cock, D. (2011)](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627). The modified version contains 2,930 rows with 79 columns describing various aspects of residential homes in Ames, Iowa. The target variable for this problem is `SalePrice`, while all other columns are treated as features.

Firstly, execute the following code snippet for data preparation (Remember to upload the dataset and import the dependent library first):

In [ ]:
def score_rmsle(X, y, model):
    """
    Encodes categorical variables using OrdinalEncoder, splits the data, trains the model,
    and returns the Root Mean Squared Log Error (RMSLE) on the validation set.

    Parameters:
    - X: pd.DataFrame, feature matrix
    - y: pd.Series or array-like, target variable
    - model: scikit-learn estimator, regression model or pipeline to train

    Returns:
    - score: float, RMSLE on the validation set
    """
    # See https://stats.stackexchange.com/questions/27750/feature-selection-and-cross-validation
    # , https://stackoverflow.com/questions/56308116/should-feature-selection-be-done-before-train-test-split-or-after
    # and https://stats.stackexchange.com/questions/2306/feature-selection-for-final-model-when-performing-cross-validation-in-machine

    # Split the data into training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(
        X.values, y, train_size=0.8, test_size=0.2, random_state=2024
    )

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    preds = model.predict(X_valid)

    # Calculate RMSLE
    score = root_mean_squared_log_error(y_valid, preds)

    return score


df = pd.read_csv("ames.csv")
X = df.copy()
y = X.pop('SalePrice')

# For simplicity, we perform label encoding for categoricals here, though it may be better to put it in pipline
# Identify categorical columns
categorical_cols = X.select_dtypes(include=["category", "object"]).columns

if len(categorical_cols) > 0:
    # Initialize OrdinalEncoder
    encoder = OrdinalEncoder(encoded_missing_value=-1)

    # Fit and transform the categorical columns
    X[categorical_cols] = encoder.fit_transform(X[categorical_cols])

To ensure reproducibility, please set all the random seeds to 2024:

### (a) K-Means Clustering as a Feature Engineering method (10%)

First, perform k-means clustering using the following parameters:

- **Features:** `FirstFlrSF`, `SecondFlrSF`, `GrLivArea`, `LotArea`, `TotalBsmtSF`
- **Number of clusters:** 6

Next, add the k-means labels and cluster distance features to your original dataset. Finally, perform regression using `RandomForestRegressor` with 10 estimators and calculate the Root Mean Log Squared Error (RMLSE) using the `scoring_rmsle()` function.

**Hint:** The `predict()` and `transform()` methods in the [k-means function](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) may be useful when generating the features.

In [ ]:
# coding your answer here.

### (b) Feature Selection Using Mutual Information (10%)

The Ames dataset contains a large number of features. Fortunately, you can identify the most promising features and apply a filtering method for feature selection.

1. **Calculate Mutual Information:** Using the dataset obtained in part (a) (with added features), report the mutual information between the target variable (`SalePrice`) and each feature in descending order using the [`mutual_info_regression()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html) function.

2. **Build a Feature Selection Pipeline:**
   - **Select Best Features:** Use the [`SelectKBest()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html) function to select the top 20 features based on mutual information.
   - **Fit the Model:** Fit a `RandomForestRegressor` with 10 estimators on the transformed dataset.
   
3. **Evaluate the Model:** Calculate the Root Mean Squared Logarithmic Error (RMSLE) on the transformed dataset using the `score_rmsle()` function.

**Hint:** Since we are working with both training and validation sets, you may find [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) useful.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

### (c) Feature Selection Using the Wrapper Method with [Boruta](https://github.com/scikit-learn-contrib/boruta_py) (10%)

Now, we will apply the wrapper method for feature selection using [Boruta](https://pdfs.semanticscholar.org/ecc2/ca3150dc4d4d8dceedab244114f191e05742.pdf). Please follow the steps below:

1. **Perform Feature Selection with Boruta:**
   - **Model Configuration:** Use `RandomForestRegressor()` with `max_depth` set to 5 and `n_estimators` set to 10 as the underlying model in Boruta.
   - **Execute Boruta:** Apply Boruta to perform feature selection on the dataset obtained in part (a).

2. **Analyze Selected Features:**
   - **Number of Features Selected:** Report how many features Boruta selects in this example.
   - **Consistency Check:** Determine whether the selected features are consistent with the top 20 features identified in part (b).

3. **Model Training and Evaluation:**
   - **Regression Model:** Use the selected features to perform regression using `RandomForestRegressor` with 10 estimators.
   - **Calculate RMSLE:** Compute the Root Mean Squared Logarithmic Error (RMSLE) using the `scoring_rmsle()` function on the validation set.

**Hint:** Since we are working with both training and validation sets, you may find [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) useful.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

## Q3: Analyzing Hospital Readmission Dataset with Interpretable Methods


Hospital readmission occurs when a patient who has been discharged from a hospital is admitted again within a specified time interval. Generally, a higher readmission rate indicates the ineffectiveness of treatment during previous hospitalizations.

Therefore, the hospital seeks your assistance in identifying patients at the highest risk of being readmitted. While doctors will make the final decision about when to discharge each patient, they hope you can build a model to highlight factors that doctors should consider when making discharge decisions. The hospital has provided relevant patient medical information. The given dataset contains the following features:

- **Prediction Target:** The prediction target is `readmitted`.
- **Number of Inpatient Visits:** Feature names like `number_inpatient` refer to the number of inpatient visits of the patient in the year preceding the encounter.
- **Diagnostic Codes:** Features containing the word `diag` indicate the diagnostic code of the illness or illnesses with which the patient was admitted. For example, `diag_1_428` means the patient's first illness diagnosis is code "428".
- **Medication Indicators:** A feature named `metformin_No` means the patient did not take the medication `metformin`. If this feature has a value of `False`, then the patient did take the drug `metformin`.
- **Medical Specialties:** Features that begin with `medical_specialty` describe the specialty of the doctor treating the patient. The values in these fields are all `True` or `False`.

**Note:** Our dataset is a cleaned and truncated version of the [Diabetes 130-US Hospitals for Years 1999-2008](https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008) dataset from the UCI Machine Learning Repository. Please refer to the above link for a detailed description of the features.

Firstly, execute the following code snippet for data preparation (Remember to upload the dataset and import the dependent library first):

In [ ]:
X = pd.read_csv('hospital.csv')
#  Removes all characters from the column names that are not letters, numbers, or underscores.
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


y = X.readmitted
X.drop(['readmitted'], axis=1, inplace=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=2024)

To ensure reproducibility, please set all the random seeds to 2024:

#### (a) Fit an Interpretable Model to Align with Medical Intuition (10%)

1. **Build a FIGSClassifier Model:**
   - **Model Construction:** Build a [`FIGSClassifier`](https://csinva.io/imodels/tree/figs.html#imodels.tree.figs.FIGSClassifier) decision rule model as a baseline.
   - **Parameters:** Set the parameters `max_rule` to 5 and `max_tree` to 1.
   - **Model Evaluation:** Calculate the accuracy of the model on the validation set.

2. **Analyze Feature Importance:**
   - **Plot Feature Importances:** Draw the feature importance plot using the [`feature_importances_`](https://csinva.io/imodels/tree/figs.html#imodels.tree.figs.FIGS.feature_importances_) attribute.
   - **Identify Key Feature:** Determine the most important feature based on the plot.

3. **Explain Readmission Decision for a Specific Patient:**
   - **Patient Identification:** Consider a patient whose data is recorded in the first row of the validation dataset with ID `20556`.
   - **Provide Explanation:** Using the rules generated from the `FIGSClassifier` model, explain to the patient the reasons for their readmission or lack thereof.
   - **Visualization:** Optionally, use a tree diagram to visualize the decision path taken by the model for this patient.

**Hint:** Ensure that the `FIGSClassifier` model is properly trained and validated before interpreting the results. Visualization tools such as tree diagrams can aid in making the decision process transparent and understandable for medical professionals.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

### (b) Enhancing Model Performance with `ExplainableBoostingClassifier` (10%)

The doctor is pleased that you have successfully convinced the patients; however, he remains concerned about the performance of the `FIGSClassifier` model you previously built.

1. **Build an `ExplainableBoostingClassifier` Model:**
   - **Model Construction:** Build a more sophisticated classifier using the [`ExplainableBoostingClassifier`](https://interpret.ml/docs/python/api/ExplainableBoostingClassifier.html).
   - **Training Parameters:** Train the model with three automatically selected interaction terms to capture complex relationships between features.

2. **Evaluate Model Performance:**
   - **Model Evaluation:** Calculate and report the accuracy of the `ExplainableBoostingClassifier` model on the validation set.

3. **Provide Model Explanations:**
   - **Global Explanation:** Display the feature importance plot to showcase which features are most influential in the model's predictions.
   - **Local Explanation:** Generate a local explanation for the patient identified in part (a) to illustrate the specific factors contributing to their readmission prediction.

4. **Consistency Analysis:**
   - **Compare Explanations:** Assess whether the explanations provided by the `ExplainableBoostingClassifier` are consistent with those from the `FIGSClassifier` model built in part (a).

**Hint:** Utilize visualization tools provided by the `interpret` library to effectively illustrate both global and local explanations.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*


### (c) Explaining Patient Readmission Using LIME and SHAP (10%)

1. **Explain the Model Using LIME:**
   - **Create a LIME Explainer:** Instantiate a [`lime_tabular.LimeTabularExplainer()`](https://lime-ml.readthedocs.io/en/latest/lime.html#module-lime.lime_tabular) using the training data.
   - **Generate Explanation:** Use the [`explain_instance()`](https://lime-ml.readthedocs.io/en/latest/lime.html#lime.lime_tabular.LimeTabularExplainer.explain_instance) method to explain the `ExplainableBoostingClassifier` model you trained in part (b) for the specific patient mentioned in (a).
   - **Visualize Local Explanation:** Draw the local explanation for this patient by displaying the five most important features influencing their readmission prediction.

2. **Explain the Model Using SHAP:**
   - **Create a SHAP Explainer:** Define the following function and instantiate a [`shap.KernelExplainer()`](https://shap.readthedocs.io/en/latest/generated/shap.KernelExplainer.html) using the function where ebm is the `ExplainableBoostingClassifier` model you trained in part (b):

     ```python
     def ebm_predict_proba(X):
         X_df = pd.DataFrame(X, columns=feature_names)
         return ebm.predict_proba(X_df)
     
     shap_explainer = shap.KernelExplainer(ebm_predict_proba, data=X_train, link="logit", feature_names=list(X_train.columns))
     ```

   - **Generate SHAP Values:** Use the above `KernelExplainer` to calculate the SHAP values for the `ExplainableBoostingClassifier` model trained in part (b) for the patient.
   - **Visualize SHAP Force Plot:** Create force plots for the patient using [`shap.force_plot()`](https://shap.readthedocs.io/en/latest/generated/shap.plots.force.html), setting the parameter `nsamples` to 100 to speed up the calculation.

3. **Compare Explanations and Assess Consistency:**
   - **Analyze Explanations:** Based on the results from both LIME and SHAP, explain why the patient was (or was not) readmitted.
   - **Consistency Check:** Discuss whether the explanations from LIME and SHAP are consistent with the interpretations from parts (a) and (b).


In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

### (d) Deploying the Model as a Service with `BentoML` (10%)

Now that the doctors are convinced you have the right data and the model overview looks reasonable, it's time to turn this into a finished product they can use.

1. **Save and Reload the Model Using BentoML:**
   - **Save the Model:** Use [`BentoML`](https://docs.bentoml.com/en/latest/index.html) to save your `ExplainableBoostingClassifier` model.
   - **Reload the Model:** Reload the saved model to ensure it has been correctly serialized and deserialized.

2. **Verify Model Consistency:**
   - **Make Inferences:** Use the reloaded model to make inferences on the patient identified in part (a).
   - **Compare Results:** Demonstrate that the inference results from the reloaded model are identical to those from the original model for the same patient.

3. **Wrap the Model as a Service:**
   - **Write a Service Script:** Develop a script to wrap your model as a [BentoML service](https://docs.bentoml.com/en/latest/guides/services.html).
   - **Start the Server:** Deploy the service by starting the BentoML server.
   - **Test the Service:**
     - **Prepare the Request:** Use the following code snippet to send a request containing the patient's data (You may need to modify the code to be compatible with your server script):
       ```python
       import json
       import requests
       import pandas as pd

       # Select the first patient from the validation set
       df_row = X_valid.iloc[0:1]
       input_series_json = df_row.to_json(orient='values')  # Example format: '[[0,1,2,...,63]]'
       input_series = json.loads(input_series_json)

       # Prepare the input data for the POST request
       input_data = {
           "input_series": input_series
       }

       # Send the POST request to the BentoML service
       response = requests.post(
           "http://127.0.0.1:8050/classify",
           json=input_data
       ).text

       print(response)
       ```
     - **Validate the Response:** Ensure that the responses from the server closely match the predictions of the original model for the patient.



In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

The following code may be useful for starting and closing the server:

In [ ]:
from pyngrok import ngrok, conf
import getpass

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/")
conf.get_default().auth_token = getpass.getpass()

# Setup a tunnel to the streamlit port 8050
public_url = ngrok.connect(8050)

print(public_url)

In [ ]:
!pgrep bentoml
!kill $(pgrep bentoml)